# Symbolic Computation with SymPy

Sometimes it is important to have exact representations of mathematical objects and manipulate them without the possibility of rounding or range errors. For example, we could want to simplify, expand, differentiate, or integrate expressions such as $(x+a)^2$ without explicit evaluation for specific numbers $x$ or $a$. Furthermore, some interesting evaluations cannot be performed by plugging in numbers, e.g., we cannot simply set $x=0$ in $\displaystyle\lim_{x\to0}\frac{\sin(x)}{x}$. In such cases, we need to consider a symbolic computation library such as SymPy.

### What you will learn

In this notebook we will use SymPy to analytically study the quantum mechanical harmonic oscillator. We will use these examples to motivate the different aspects of symbolic calculations, so while there will be references to quantum mechanics, we will mainly cover topics of calculus. No knowledge of quantum mechanics is required for this notebook.

### Warning
DO NOT run "all cells" in this notebook. There is a cell in here that will lead to errors and can freeze your system. 

*&#169; Tobias Hartung, University of Bath 2021-2022. This problem sheet is copyright of Tobias Hartung, University of Bath. It is provided exclusively for educational purposes at the University and is to be downloaded or copied for your private study only. Further distribution, e.g. by upload to external repositories, is prohibited.*

## Basics of SymPy

Before we will jump into the main example of the notebook, let us quickly look at the basics.

For most applications, rounding errors incurred during computation are not relevant. For example, we could compute $x=\frac{1}{\sqrt{8}}$ with NumPy and compare $x^2$ to the correct value $0.125$.

In [ ]:
import numpy as np

x = 1/np.sqrt(8)
y = x**2
print(y)

This shows an error in the 17th digit. Such errors are usually fine as there are many other error sources that have far larger impact. However, if a computation is numerically instable (e.g., many chaotic dynamical system like weather forecast), then these errors will accumulate exponentially fast. 

#### Example - Logistic model

Consider the logistic model 

$$N(t+1)=4N(t)(1-N(t))\text{ with }t\in\mathbb{N}_0.$$ 

In this model $N(t)\in[0,1]$ is a population at time $t$. The population $N$ grows proportionally to the current population size if $N(t)\approx0$ and which starves if $N(t)\approx1$ because of a lack of resources. 

Let us assume that $N(0)$ is computed to be $N(0)=(1/\sqrt{8})^2=1/8=0.125$ and let us compare two ways of evaluating $N(0)$.

a. The correct value `N_true = 0.125`

b. The numerically obtained value `N_error = (1/np.sqrt(8))**2` $\approx 0.12499999999999997$

Let us see what happens to the solution $N(t)$ with these two values for $N(0)$ that only differ up to rounding errors. Specifically, let us compute the difference $N_{true}(t)$ and $N_{error}(t)$ as $t$ grows.

In [ ]:
N_true = 0.125
N_error = (1/np.sqrt(8))**2

max_error = abs(N_error-N_true)

print('maximal error at t=0: ',max_error)

for index in range(6):
    for t in range(10):
        N_true = 4*N_true*(1-N_true)
        N_error = 4*N_error*(1-N_error)
        max_error = max(max_error, abs(N_error-N_true) )
    print('maximal error at t='+str(10*(index+1))+': ', max_error)


As we can see, although the error is minor to begin with, it only requires 40 time-steps for the error to become noticeable. Furthermore, at some point between time-step 50 and 60, the error was roughly $0.97$. Given that the entire domain is the interval $[0,1]$, that means that the two values were at opposite ends of the domain. The two values were $N_{true}(58)\approx0.015$ and $N_{error}(58)\approx0.985$. They could hardly be further away from each other in the model. Thus, being able to compute $N(0)$ error-free would greatly improve the numerics of such a model. 

This is where SymPy enters the picture.

In [ ]:
import sympy

x = 1/sympy.sqrt(8)
print('x =', x)

y = x**2
print('y =', y)

Here, we can see that SymPy keeps the exact value for $\sqrt2$ and thus computes `y` error-free. Had we used SymPy in the example above, we wouldn't even need to specify the initial value to compute the next values (we can do this later) and obtain the following result. We will discuss the SymPy functions used in this example as we progress in the notebook.

In [ ]:
n = sympy.symbols("n")
N_true = n
N_error = n

max_error = sympy.Abs(N_error-N_true)

print('maximal error at t=0: ',max_error)

for index in range(10):
    for t in range(10):
        N_true = 4*N_true*(1-N_true)
        N_error = 4*N_error*(1-N_error)
        max_error = max(max_error, sympy.Abs(N_error-N_true))
    print('maximal error at t='+str(10*(index+1))+': ', max_error)


In fact, this example is not far from how Lorenz discovered chaos. He used to study weather by solving certain differential equations numerically. One day he wanted to reproduce some earlier numerical results. To save time, he started his simulation in the middle of the original simulation using rounded data. The rouding difference was tiny compared to the studied effects and the general consensus at the time would have been that no practical effect should be observed. To his surprise, the second simulation showed wildly different weather predictions. As we know now, these rounding errors can have large impact, and thus the field of chaos theory has been born. In such circumstances, symoblic computation can make a major difference.

### Symbols

Symbolic computation earns its salary by being able to work with generic variables. These can be defines using the `symbols` function which takes a string of symbols split by spaces. Once symbols are defined, we can work with them algebraically.

In [ ]:
# define symbols x, y, and alpha
x, y, alpha = sympy.symbols("x y alpha")

# define an expression with them
expr = x*(x+alpha*y)+x
print('expression =', expr)

# we can expand the expression
expanded_expr = sympy.expand(expr)
print('expanded expression =', expanded_expr)

# we can do basic arithmetic
expr_2 = expanded_expr - x
print('new expression =', expr_2)

# and we can factor
factored_expr_2 = sympy.factor(expr_2)
print('factored expression =', factored_expr_2)

# and of course simplify
expr_3 = expr_2 - factored_expr_2
simplified_expr_3 = sympy.simplify(expr_3)
print('third expression =', expr_3)
print('simplified third expression =', simplified_expr_3)

It is important to note that symbols are used in two ways. (a) there is the variable name in Python that you type to call something like `x` in `x=2` and (b) there is the SymPy object that is printed as the symbol `x`. As such, the variable name you type in Python bears no inherent relation to the Sympy symbol, just like after assigning `x=2` the print command `print(x)` will put `2` on the screen and not `x`. Therefore, it is imperative to keep in mind that there is a Python variable that we use to address objects in memory with (here `x` as in `x=2`) and a SymPy symbol which is an object in memory (such as `2` in `x=2`). 

Ideally, you should store the SymPy symbol `x` in the Python variable `x` to avoid confusion. Similarly, you should avoid re-defining Python variable that store SymPy symbols as this would be highly confusing, as well. To illustrate this point, consider the following example. 

In [ ]:
a,b,something = sympy.symbols("b a else")
print(a)
print(b)
print(something)

Furthermore, expressions do not update when you update a variable.

In [ ]:
c = sympy.symbols("c")
expr = c+1
c = 2
print(expr)
c = x
print(expr)

In fact, SymPy expressions are immutable, so if you want to replace symbols with values (or other symbols), then you need to use `.subs()` to substitute them and assign the result to a new expression (you may overwrite the existing expression).

In [ ]:
c, x = sympy.symbols("c x")
expr = c+1
print(expr)
# substitute c for x
expr = expr.subs(c,x)
print(expr)
# substitute x for 2
expr = expr.subs(x,2)
print(expr)

Finally, before jumping into some more interesting symbolic computations, it is important to note that Python will evaluate expressions before SymPy if Python knows how to do it. Thus, if we wanted to define the fraction 1/2 in SymPy, we cannot write `1/2` directly as Python will compute this to be the floating point number `0.5` first.   

In [ ]:
# Python jumping the gun
expr = x + 1/2
print(expr)

# we could define 1 and 2 as Sympy integers first 
expr = x + sympy.Integer(1)/sympy.Integer(2)
print(expr)

# it's sufficient to have one of them a SymPy object to stop Python from doing its thing
expr = x + sympy.Integer(1)/2
print(expr)

# or we could define 1/2 as a SymPy rational
expr = x + sympy.Rational(1,2)
print(expr)

Of course, this requires us to know what the corresponding SymPy structure should be (by that I mean that we need to know the commands `sympy.Integer` and `sympy.Rational` in the example above). Luckily, we don't need to worry about this too much as for most structures, SymPy will be able to guess what you mean and do the right thing if you call `sympify`. 

In [ ]:
# we can convert a string into an expression
expr = sympy.sympify("x + 1/2 - sqrt(y)")
print(expr)

Similarly, we can convert SymPy numbers to floats with `evalf()` and expressions to functions with `lambdify`.

In [ ]:
print('pi =', sympy.pi)
print('pi is approximately', sympy.pi.evalf())
print('The first 50 digits of pi are', sympy.pi.evalf(50))

print()
# let's recall the expression
print('The expression was', expr)
function = sympy.lambdify([x,y], expr)
print('Its value at x=1 and y=4 is', function(1,4))

### Printing

While `print()` works reasonably well on not too complicated expressions, non-trivial expressions can become cumbersome to read (especially when they are not simplified). For example, the arc-length between $(x_1,y_1)$ and $(x_2,y_2)$ of the ellipse $\frac{x^2}{a^2}+\frac{y^2}{b^2}=1$ with $y_1,y_2>0$ is given by

In [ ]:
a,b,x1,x2,z = sympy.symbols("a b x1 x2 z")
s = sympy.Integral(b*sympy.sqrt(1-(1-a**2/b**2)*sympy.sin(z)**2),(z,sympy.acos(x2/a),sympy.acos(x1/a)))
print(s)

This is already sufficiently hard to decipher if it is given as a printed result of a computation. But we can make this more usable with the pretty print `pprint()` command. It usually defaults to unicode but this can be changed.

In [ ]:
sympy.init_printing()               # make pprint available
print('default printing:')
sympy.pprint(s)                     # use default options
print('')
print('forced unicode printing:')
sympy.pprint(s,use_unicode=True)    # force unicode
print('')
print('force not unicode printing:')
sympy.pprint(s,use_unicode=False)    # force ascii if unicode is not available

You can even make SymPy generate $\LaTeX$ output to copy into your documents. If we copy the $\LaTeX$ output generated by

In [ ]:
print(sympy.latex(s))

directly into this Notebook here, we obtain

$$\int\limits_{\operatorname{acos}{\left(\frac{x_{2}}{a} \right)}}^{\operatorname{acos}{\left(\frac{x_{1}}{a} \right)}} b \sqrt{- \left(- \frac{a^{2}}{b^{2}} + 1\right) \sin^{2}{\left(z \right)} + 1}\, dz$$

Similarly, there is MathML should you require output for web use through

```
from sympy.printing.mathml import print_mathml
print_mathml(s)
```

Here we will not output the MathML since it is essentially unreadable unless parsed by a server that is able to understand the markup language.

## The Harmonic Oscillator

Our main example for this notebook is the harmonic oscillator. For interested students, there will be a more in-depth explanation of the physical background at the end of the notebook (after the "Check your understanding" answers). For this section, we will simply use the the formulas on faith but if you are interested in further discussion of this, I encourage you to skip ahead or read on after the "Check your understanding" answers.


### Hermite Polynomials

To describe the harmonic oscillator, physicists use the Hermite polynomials $H_n(x)$. The first five Hermite polynomials are

1. $H_0(x)=1$
1. $H_1(x)=2x$
1. $H_2(x)=4x^2-2$
1. $H_3(x)=8x^3-12x$
1. $H_4(x)=16x^4-48x^2+12$

The Hermite polynomials satisfy the equation

$$H_n''(x) - 2\,x\cdot H_n'(x) + 2\,n\cdot H_n(x)=0\qquad   (n=0,1,2,\dots).$$

We can verify this using SymPy for the first five. To take derivatives, we can use `sympy.Derivative()`.

In [ ]:
x = sympy.symbols("x")

h0 = sympy.sympify(1)
h1 = 2*x
h2 = 4*x**2 - 2
h3 = 8*x**3 - 12*x
h4 = 16*x**4-48*x**2+12

LHS_0 = sympy.Derivative(h0,(x,2)) - 2*x*sympy.Derivative(h0,(x,1)) + 2*0*h0
LHS_1 = sympy.Derivative(h1,(x,2)) - 2*x*sympy.Derivative(h1,(x,1)) + 2*1*h1
LHS_2 = sympy.Derivative(h2,(x,2)) - 2*x*sympy.Derivative(h2,(x,1)) + 2*2*h2
LHS_3 = sympy.Derivative(h3,(x,2)) - 2*x*sympy.Derivative(h3,(x,1)) + 2*3*h3
LHS_4 = sympy.Derivative(h4,(x,2)) - 2*x*sympy.Derivative(h4,(x,1)) + 2*4*h4

print('LHS for h0 =', LHS_0.doit())
print('LHS for h1 =', LHS_1.doit())
print('LHS for h2 =', LHS_2.doit())
print('LHS for h3 =', sympy.simplify(LHS_3.doit()))
print('LHS for h4 =', sympy.simplify(LHS_4.doit()))


Here, the `.doit()` forces SymPy to perform the derivative as otherwise it would try to avoid any costly operations to improve performance. Similarly, the `(x,2)` and `(x,1)` arguments tell SymPy to take two or one derivative with respect to `x`. You could stack multiple derivatives in there, e.g., `sympy.Derivative(expr,(x,2),(y,5),(z,1))` would take two `x`-derivatives, five `y`-derivatives, and one `z`-derivative of the expression `expr`. 

Returning to the general form

$$H_n''(x) - 2\,x\cdot H_n'(x) + 2\,n\cdot H_n(x)=0\qquad   (n=0,1,2,\dots),$$

we need to solve an ordinary differential equation which, luckily, we can do analytically:

$$H_n(x) = (-1)^n e^{x^2} \frac{d^n}{dx^n} e^{-x^2}.$$

This still requires taking high order derivatives. Trying to compute these numerically is still difficult and will not provide us with a functional form. But SymPy can compute these polynomials exactly. Importantly, we can even take the $n$th derivative using `sympy.Derivative(expr,(x,n))`.

In [ ]:
n,x = sympy.symbols("n x")

# Hermite polynomials
Hn = (-1)**n * sympy.exp(x**2) * sympy.Derivative(sympy.exp(-x**2),(x,n))

# e.g.,
print("H0 = ",Hn.subs(n,0).doit())
print("H1 = ",Hn.subs(n,1).doit())
print("H2 = ",Hn.subs(n,2).doit())
print("H3 = ",sympy.expand( Hn.subs(n,3).doit() ))
print("H4 = ",sympy.expand( Hn.subs(n,4).doit() ))
print("H5 = ",sympy.expand( Hn.subs(n,5).doit() ))

The Hermite polynomials can also be computed using a recurrance relation

$$H_k(x)=2xH_{k-1}(x)-2(k-1)H_{k-2}(x)$$

with $H_0(x)=1$ and $H_1(x)=2x$. Using this, we can also compute high order Hermite polynomials without having to compute the derivatives explicitly. For example,

$$
H_5(x) = 2xH_4(x) -2\cdot4\cdot H_3(x) = 2x(16x^4-48x^2+12) - 8(8x^3-12x) = 32x^5-96x^3+24x -64x^3+96x = 32x^5-160x^3+120x.
$$

However, we have used quite a bit of simplifications in this computation. SymPy will need to do that as well. Otherwise, the computation can become unmanageable, as the following example computation of $H_{15}$ shows.

In [ ]:
H_list = [sympy.sympify(1),2*x] # initialise list of Hermite polynomials

for k in range(2,16):
    H_list.append( 2*x*H_list[k-1] - 2*(k-1)*H_list[k-2] )

print(H_list[15])

As we can see, this output is massive. This needs to be simplified in order to use it properly. Best to compute them with simplifications in each step. 

In [ ]:
H = [sympy.sympify(1),2*x] # initialise list of Hermite polynomials

for k in range(2,16):
    H.append( sympy.simplify(2*x*H[k-1] - 2*(k-1)*H[k-2]) )

print(H[15])

This is much better. And we can see that this improves performance as well. For example, we can compare the time it takes to evaluate the two versions at $x=\pi$.

In [ ]:
import timeit

print("not simplified: ",timeit.timeit(lambda : H_list[15].subs(x,sympy.pi).evalf(), number=1))
print("simplified: ",timeit.timeit(lambda : H[15].subs(x,sympy.pi).evalf(), number=1))
print("derivative defined: ",timeit.timeit(lambda : Hn.subs(n,15).doit().subs(x,sympy.pi).evalf(), number=1))

Note, it is about 5000 times faster to work with the simplified version than the non-simplified. Even the "slow" definition using derivatives is still 500 times faster than without simplifying. 

### Physics application of Hermite Polynomials

In physics, the Hermite polynomials $H_n(x)$ appear as part of fundamental functions

$$\psi_n(x)=\frac{1}{\pi^{\frac{1}{4}}\sqrt{2^n n!}}e^{-\frac{x^2}{2}}H_n(x)$$

that describe the harmonic oscillator. We can use the Hermite polynomials computed above to define these functions $\psi_n$.

In [ ]:
psi_n = Hn * sympy.exp(-x**2 / 2) / (sympy.pi**(1/sympy.sympify(4)) * sympy.sqrt(2**n * sympy.factorial(n)) )

Just like the Hermite polynomials, the $\psi_n$ satisfy a recurrance relation 

$$\psi_{n+1}(x)=\frac{x\psi_n(x)-\frac{d}{dx}\psi_n(x)}{\sqrt{2(n+1)}}.$$

which is related to physical processes. Hence, we can compute those $\psi_n(x)$ iteratively starting from 

$$\psi_0(x)=\frac{1}{\pi^{\frac{1}{4}}\sqrt{2^0 0!}}e^{-\frac{x^2}{2}}H_0(x)=\frac{1}{\pi^{\frac{1}{4}}}e^{-\frac{x^2}{2}}.$$

In [ ]:
psi = [sympy.exp(-x**2 / 2) / (sympy.pi**(1/sympy.sympify(4)))] # initialise with psi_0

for k in range(1,16):
    psi.append( sympy.simplify( ( x*psi[k-1] - sympy.Derivative(psi[k-1]).doit() )/sympy.sqrt(2*k) ) )

# check the expressions coincide with previous psi_n by checking their difference is 0
print(sympy.simplify(psi[15]-psi_n.subs(n,15).doit()))

The functions $\psi_n(x)$ are physically interesting because they represent fundamental physical situations with different energies. You can think of the physical relevance here in terms of Bohr's atom model.

![title](Bohr_atom_model.svg)

Source: https://en.wikipedia.org/wiki/Bohr_model

These different $n=1$, $n=2$, $n=3$ (and so on) layers go a long way of describing basic chemistry which you may have seen during A-levels. Even the energy difference is indicated in this picture from Wikipedia which highlights that this is a very fundamental aspect of the theory. 

Without going into physical detail, we obtain the energy $E_n$ that corresponds to $\psi_n(x)$ by evaluating

$$E_n=\frac{1}{2}\int_{-\infty}^\infty x^2\psi_n(x)^2-\psi_n(x) \psi_n''(x)dx.$$

We can compute this with `sympy.Integral()` where $\infty$ is given by `sympy.oo`.

In [ ]:
En = sympy.Integral(x**2 * psi_n**2 - psi_n * sympy.Derivative(psi_n,(x,2)) ,(x,-sympy.oo,sympy.oo)) / 2
En

Although it is possible to show analytically that $E_n=\frac{2n+1}{2}$, this looks sufficiently horrible that we do not wish to compute this by hand. In fact, this is sufficiently horrifying that even SymPy needs some manual encouragement to properly evaluate this without substituting $n$. Since the techniques to make SymPy do this properly are very technical, they would go far beyond what is reasonable necessary for an introduction to symbolic computations and we will therefore not discuss them here in detail. So instead, let's use SymPy to compute some values here and verify the claim $E_n=\frac{2n+1}{2}$ for some specific values of $n$.

In [ ]:
print("E_0 =",En.subs(n,0).doit())
print("E_1 =",En.subs(n,1).doit())
print("E_10 =",En.subs(n,10).doit())

It is interesting to note that this integral

$$E_n=\int_{-\infty}^\infty \psi_n(x)\ \frac{x^2\psi_n(x)-\psi_n''(x)}{2}dx.$$

is actually a dot-product 

$$\langle f,g\rangle = \int_{-\infty}^\infty f(x)g(x)dx$$

with $f(x)=\psi_n(x)$ and $g(x)=\frac{1}{2}(x^2\psi_n(x)-\psi_n''(x))$ in this vector space of functions that describes quantum mechanics. Given that this is a dot-product with functions acting as vectors, we can generalise the ideas from MA10236 of orthogonality and we find that two functions are "orthogonal" if $\int_{-\infty}^\infty f(x)g(x)dx=0$ and the "length of the vector $f$" is 

$$\|f\| = \sqrt{\int_{-\infty}^\infty f(x)^2dx}.$$

If we consider our functions $\psi_n$, we find that they are mutually orthogonal and of unit length.

In [ ]:
m = sympy.symbols("m")
psi_m = psi_n.subs(n,m) # substitute n for m to define psi_m

psi_n_dot_psi_m = sympy.Integral(psi_n * psi_m ,(x,-sympy.oo,sympy.oo))
sympy.pprint(psi_n_dot_psi_m)

print() # print empty line

print("length of psi_0 =",sympy.sqrt(psi_n_dot_psi_m.subs(n,0).subs(m,0).doit()))
print("length of psi_1 =",sympy.sqrt(psi_n_dot_psi_m.subs(n,1).subs(m,1).doit()))
print("length of psi_10 =",sympy.sqrt(psi_n_dot_psi_m.subs(n,10).subs(m,10).doit()))

print() # print empty line

print("dot product of psi_0 and psi_1 =",psi_n_dot_psi_m.subs(n,0).subs(m,1).doit())
print("dot product of psi_5 and psi_10 =",psi_n_dot_psi_m.subs(n,5).subs(m,10).doit())

This means that we can use these functions as a basis for an infinite dimensional vector space. Every function $f$ in this space can be written as a linear combination

$$f=\alpha_0\psi_0+\alpha_1\psi_1+\ldots$$

where the $\alpha_n$ are given by 

$$\alpha_n=\int_{-\infty}^\infty f(x)\psi_n(x)dx.$$

Computing these integrals for $\alpha_n$ is very common in physics since the function $f$ can be used to describe a more general physical system. In that case, we usually call this function $\psi$ rather than $f$.

Let's say our function $\psi$ is 

$$\psi(x)=\psi_0(x)/3+2\psi_1(x)/3.$$

Then 

$$\alpha_0=\int_{-\infty}^\infty \psi(x)\psi_0(x)dx$$

can be computed as

In [ ]:
psi = psi_n.subs(n,0)/3 + 2*psi_n.subs(n,1)/3

alpha_n = sympy.Integral(psi*psi_n,(x,-sympy.oo,sympy.oo))
sympy.pprint(alpha_n.subs(n,0).doit())

More generally, we could consider

$$\psi(x)=\pi^{-\frac{1}{4}}e^{-|x|}.$$

and compute $\alpha_0$ as

In [ ]:
psi = sympy.pi**(-sympy.sympify(1)/4) * sympy.exp(-sympy.Abs(x))

alpha_n = sympy.Integral(psi*psi_n,(x,-sympy.oo,sympy.oo))
sympy.pprint(alpha_n.subs(n,0).doit())

Although it seems to be able to compute it, it has this function `erf` in the result which is not analytically computable. So in general this integral for $\alpha_n$ still needs to be approximated. Unfortunately, the mathematical methods required to do this for a model with $x\in\mathbb{R}$ go beyond the scope of this notebook. But if we had a model where $x$ only ranges over a bounded interval, then we can do a bit more here.

### A microwave example

Another example of a quantum system is a "microwave". In this case, our functions $\psi_n$ will be called $\phi_n$ to avoid confusion with the $\psi_n$ used in the previous section (the harmonic oscillator example). These new $\phi_n$ are given by

$$\phi_n(x)=\sqrt{\frac{2}{\pi}}\cos((2n+1)x)\qquad\text{for}\qquad -\frac{\pi}{2}<x<\frac\pi2,\ n=0,1,2,\ldots.$$

A general state that could describe the microwave can be a linear combination of the $\phi_n$. For example, we may consider

$$\phi(x)=c\exp\left(-\frac{1}{\frac{\pi^2}{4}-x^2}\right)$$

where 

$$c=\frac{1}{\sqrt{\int_{-\pi/2}^{\pi/2}\exp\left(-\frac{2}{\frac{\pi^2}{4}-x^2}\right)dx}}\approx1.06.$$

It is important to note that for physical reasons the wave function $\phi$ must satisfy $\phi(\pm\pi/2)=0$ but we cannot actually evaluate $\phi$ at these points because we have a division by zero in the exponent. This is symbolized by the outcome `NaN` (meaning "not a number") when we try to evaluate $\phi$ at $\pi/2$ for example.

In [ ]:
# define function phi
c = sympy.symbols("c")
phi = c* sympy.exp(-1/((sympy.pi/2)**2-x**2))

# actual value for c evaluated as float for later
c_val = 1/sympy.sqrt(sympy.Integral(sympy.exp(-2/((sympy.pi/2)**2-x**2)),(x,-sympy.pi/2,sympy.pi/2))).evalf()

phi.subs(x,sympy.pi/2)

However, we can check this by taking the one-sided limits through `sympy.limit()`.

In [ ]:
# compute limits phi(+/- pi/2)
print("phi(-pi/2) =",sympy.limit(phi,x,-sympy.pi/2,"+"))
print("phi(pi/2) =",sympy.limit(phi,x,sympy.pi/2,"-"))

Note the use of `"+"` and `"-"` to tell SymPy which side to take the limit from. If we ignore this argument, then it will automatically use the two-sided limit.

In [ ]:
print("limit of sin(x)/x for x->0 =",sympy.limit(sympy.sin(x)/x,x,0))

Let us now define the functions $\phi_n$ explicitly.

In [ ]:
# define functions phi_n
phi_n = sympy.sqrt(2/sympy.pi)*sympy.cos((2*n+1)*x)

Let us now compute the value $\alpha_0$. The corresponding expressing in this case is

$$\alpha_0=\int_{-\pi}^\pi \phi(x)\phi_0(x)dx=\int_{-\pi}^\pi c\sqrt{\frac{2}{\pi}}\exp\left(-\frac{1}{\frac{\pi^2}{4}-x^2}\right)\cos(x)dx$$

which cannot be solved analytically. 

##### SymPy will struggle to execute the following cell. Depending on how SymPy is set up it will either execute forever, time-out and print a representation of the integral with some integral still in place, or simply exit with a cryptic error because it is interrupted at some random point.

##### Thus, execute the following cell at your own risk and be prepared to restart the notebook with kernel iterrupt. 

In [ ]:
sympy.Integral(phi * phi_n.subs(n,0),(x,-sympy.pi/2,sympy.pi/2)).doit()

Instead, we can approximate the result by taking a (Taylor) series expansion

$$\phi(x)=\phi(x_0) + \phi'(x_0)(x-x_0) + \frac{\phi''(x_0)}{2}(x-x_0)^2 + \ldots + \frac{\phi^{(n-1)}(x_0)}{(n-1)!}(x-x_0)^{n-1}+O((x-x_0)^{n})$$

of $\phi$ with `series(x,x0,n)` where `x` is the variable to expand in, `x0` the center point, and `n` the order of the expansion. For example, let us compute the series expansion of $\phi$ to order $3$.

In [ ]:
def approx(order):
    return (phi).series(x,0,order)

approx(3)


Note that the order $3$ is not included in the series expansion. So if you wish to contain the $x^n$ term and have $O(x^{n+1})$ in the expansion, then you need to call `series(x,x0,n+1)`. The order $O(x^3)$ term can also be suppressed using `removeO()`.

In [ ]:
approx(3).removeO()

Now, we can plug this into our integrals and approximate $\alpha_0$ to higher and higher accuracy by increasing the order in `approx`. However, this series computation with all those exponentials including $\pi$ will be very slow. 

In [ ]:
# re-defining approx to remove O() term
def approx(order):
    return (phi.evalf()).series(x,0,order).removeO()

def approx_alpha_0(order):
    return sympy.Integral( (approx(order)*phi_n.subs(n,0)) ,(x,-sympy.pi/2,sympy.pi/2)).doit()

print("order 1 approximation of alpha_0 =",approx_alpha_0(1))
print()
print("order 3 approximation of alpha_0 =",approx_alpha_0(3))
print()
print("order 5 approximation of alpha_0 =",approx_alpha_0(5))

If we were to go to higher and higher orders, the value for $\alpha_0$ will approach $\alpha_0\approx0.991$. This means that about 98% of $\phi$ are adequately approximated using $\phi_0$ (see physics background at the end of the lecture for a bit more detail on why this is).

We can also visualise what is happening here. First let us compare $\phi$ to $\phi_0$, $\phi_1$, $\ldots$, $\phi_4$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# turn phi and phi_0 into functions
func_phi = sympy.lambdify(x, phi.subs(c,c_val))
func_phi0 = sympy.lambdify(x, phi_n.subs(n,0).subs(c,c_val))
func_phi1 = sympy.lambdify(x, phi_n.subs(n,1).subs(c,c_val))
func_phi2 = sympy.lambdify(x, phi_n.subs(n,2).subs(c,c_val))
func_phi3 = sympy.lambdify(x, phi_n.subs(n,3).subs(c,c_val))
func_phi4 = sympy.lambdify(x, phi_n.subs(n,4).subs(c,c_val))

grid = np.linspace(-np.pi/2,np.pi/2,1001,endpoint=False)[1:]   # set up x-value grid for plotting without endpoints (avoid division by zero)

plt.figure(figsize=[15,10])
plt.plot(grid,func_phi(grid),"b-",label="$\phi$")
plt.plot(grid,func_phi0(grid),"r--",label="$\phi_0$")
plt.plot(grid,func_phi1(grid),"g--",label="$\phi_1$")
plt.plot(grid,func_phi2(grid),"c--",label="$\phi_2$")
plt.plot(grid,func_phi3(grid),"m--",label="$\phi_3$")
plt.plot(grid,func_phi4(grid),"y--",label="$\phi_4$")
plt.legend(fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.show()

Thus, we can see that $\phi$ and $\phi_0$ are fairly close to each other while the others differ quite a lot and the difference between $\phi$ and $\phi_n$ is getting larger as $n$ grows. This shows why $\phi$ and $\phi_0$ are about 98% the same.

Similarly, we can visualise the approximations of the integrand, i.e., the product of the order $n$ series expansion of $\phi$ times $\phi_0$. 

In [ ]:
func_phi_phi0 = sympy.lambdify(x, (phi*phi_n.subs(n,0)).subs(c,c_val))
func_approx_3 = sympy.lambdify(x, (approx(3)*phi_n.subs(n,0)).subs(c,c_val))
func_approx_5 = sympy.lambdify(x, (approx(5)*phi_n.subs(n,0)).subs(c,c_val))
func_approx_7 = sympy.lambdify(x, (approx(7)*phi_n.subs(n,0)).subs(c,c_val))
func_approx_9 = sympy.lambdify(x, (approx(9)*phi_n.subs(n,0)).subs(c,c_val))
func_approx_11 = sympy.lambdify(x, (approx(11)*phi_n.subs(n,0)).subs(c,c_val))
func_approx_13 = sympy.lambdify(x, (approx(13)*phi_n.subs(n,0)).subs(c,c_val))

# set up x-value grid for plotting without endpoints (avoid division by zero)
grid = np.linspace(-np.pi/2,np.pi/2,1001,endpoint=False)[1:]   

plt.figure(figsize=[15,10])
plt.plot(grid,func_phi_phi0(grid),"b-",label="$\phi\cdot\phi_0$")
plt.plot(grid,func_approx_3(grid),"r--",label="approx order 3")
plt.plot(grid,func_approx_5(grid),"g--",label="approx order 5")
plt.plot(grid,func_approx_9(grid),"c--",label="approx order 9")
plt.plot(grid,func_approx_11(grid),"m--",label="approx order 11")
plt.plot(grid,func_approx_13(grid),"y--",label="approx order 13")
plt.legend(fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.show()

Thus, as we increase the order of the series expansion, these approximations of the integrand become very close to the original integrand with just a little deviation visible around $\pi/2$ and $-\pi/2$. Fundamentally, this is why this approach to approximating $\alpha_0$ works and we could use it for arbitrary $\alpha_n$ and arbitrary wave functions $\phi$. 

# Check your understanding

##### Question 1
How do you define a symbolic variable?

##### Question 2
How do you assign a different value to a variable inside an expression?

##### Question 3
How do you change a number/expression into their SymPy equivalent format?

##### Question 4
How do you change SymPy numbers into floating point numbers?

##### Question 5
How do you change SymPy expressions into functions? 

##### Question 6
How do you force SymPy to perform complicated (potentially time consuming) operations such as derivatives, integrals, or limits? 

```























```

# Answers
Q1: `sympy.symbols` 
Q2: `expr.subs(x,y)`
Q3: `sympy.sympify()`
Q4: `expr.evalf()`
Q5: `sympy.lambdify()`
Q6: `expr.doit()`

## More on Physics Background

Our main example for this notebook is the harmonic oscillator. It is a crude physical model that describes two interacting particles, e.g., the two atoms in a diatomic molecule (like hydrogen or oxygen). Let us think of it in terms of a hydrogen molecule. The harmonic oscillator then describes the bond between the two atoms as well as the vibrations in the distance between the two molecules. In quantum mechanics this is described in terms of so-called wave functions which provide probability distributions. If we set all physical constants to one and assume that the molecule is not excited, i.e., it is in its energetically most favourable state, then the length of the bond (that is, the distance between the two hydrogen atoms in the molecule) is a Gaussian centered at the expected length of the bond. If we say this expected length of the bond corresponds to $x=0$, then the "wave function" for this state of the molecule is 

$$\psi_0(x)=\pi^{-\frac{1}{4}}e^{-\frac{x^2}{2}}.$$

If we put energy into the molecule, then we can excite the bond which changes this wave function. Due to quantum mechanics being weird, there are certain natural wave functions that correspond to the basic excitations of the molecule. 

Here you can think of a low brass instrument like a trumpet. The musician can change the musical note of the instrument plays by changing the amount of energy they put into the instrument (changing how they exhale). But there are only discrete notes that can be achieved as such. For other notes, the musician needs to press a valve. 

The excitations of our hydrogen molecule are similar where it can only have certain discrete energetic states. The general wave function for the $n$th state is 

$$\psi_n(x)=\frac{1}{\pi^{\frac{1}{4}}\sqrt{2^n n!}}e^{-\frac{x^2}{2}}H_n(x)$$

where $H_n(x)$ is the $n$th Hermite polynomial.

These $\psi_n(x)$ are related through physical processes. 

Using physics, there is another way of computing the Hermite polynomials because the wave functions are related by physical processes. If we put energy into the molecule, e.g., through exposure to light, some of this energy can get absorbed. This absorption means that the molecule gets excited and the corresponding wave function changes. For example, we could be in the state $\psi_0$ and get excited into the state $\psi_1$. This operation is physically described by an operator $a^\dagger=\frac{x-\frac{d}{dx}}{\sqrt2}$. The wave functions then satisfy

$$\sqrt{n+1}\psi_{n+1}(x)=a^\dagger\psi_n(x)=\frac{x\psi_n(x)-\frac{d}{dx}\psi_n(x)}{\sqrt2}.$$

Similarly, there is an operator $a=\frac{x+\frac{d}{dx}}{\sqrt2}$ which describes the loss of energy. Physically, this operation describes what happens when the molecule reduces its own energy by emitting light. Hence, this is the basis of effects like luminescence. 

Mathematically, this means that the wave functions satisfy

$$\sqrt{n}\psi_{n-1}(x)=a\psi_n(x)=\frac{x\psi_n(x)+\frac{d}{dx}\psi_n(x)}{\sqrt2}.$$

As these two operators $a$ and $a^\dagger$ translate between different energetic states of the molecule, they can be used to compute the energy of the state $\psi_n$. In fact, the operator

$$H = a^\dagger a+\frac12=\frac{x^2-\frac{d^2}{dx^2}}{2}$$

is called the Hamiltonian operator which represents the energy stored in the molecule. The $\psi_n$ are "eigenvectors" to this Hamiltonian, i.e., we have an eigenvalue equation

$$H\psi_n=E_n\psi_n$$

for the energy levels $E_n$ which can also be computed as

$$E_n=E_n\langle\psi_n,\psi_n\rangle=\langle\psi_n, H\psi_n\rangle$$

by using the facts that the $\psi_n$ actually form an orthogonal basis of the vector space quantum mechanics is modeled on and that the $\psi_n$ have norm 1 in this space. This inner product for functions is given by

$$\langle f,g\rangle = \int_{-\infty}^\infty f(x)g(x)dx.$$

Hence, we obtain the energy $E_n$ stored in the molecule if it is in the state $\psi_n$ by evaluating

$$E_n=\frac{1}{2}\int_{-\infty}^\infty x^2\psi_n(x)^2-\psi_n(x) \psi_n''(x)dx.$$

##### Linear combinations of $\psi_n$

A physical system can be in a linear combination 

$$\psi=\alpha_0\psi_0+\alpha_1\psi_1+\ldots$$

of the functions $\psi_n$. This is called "superposition. The coefficients $\alpha_n$ are then given by

$$\alpha_n=\langle f,\psi_n\rangle=\int_{-\infty}^\infty f(x)\psi_n(x)dx.$$

A weird property of quantum mechanics is the fact that the system can happily be described by a function $\psi$ which is a linear combination of the $\psi_n$ whenever it is not oberved. But if it is observed, it will randomly choose one of the $\psi_n$ and become precisely that chosen $\psi_n$. This sudden change of the system being described by a function $\psi$ before measurement and $\psi_n$ after measurement is called "the collapse of the wave function" and a fundamental aspect of why quantum mechanics are so non-intuitive. The interesting thing is though that the probability of the system choosing $\psi_n$ upon observation is exactly $\alpha_n^2$. 

In the microwave section, we can compute $\alpha_0\approx0.991$ by going to larger values of the approximation order. This means that $\alpha_0^2\approx0.98$, so this wave function $\phi$ is about 98% of the lowest energy state $\phi_0$ for this model of a microwave. This is in fact something physically reasonable as interaction with the environment or even manufacturing tolerances could introduce such variation. 